In [1]:
#|default_exp utils
from l402.payment_providers import *

# utils

> Contains a mix of dev utilities like running FastAPI in a notebook or creating test wallets.

## Onchain

In [ ]:
#| export
import os
from cdp import *

In [ ]:
#| export

def create_test_wallet(fund=True, chain='base-sepolia'):
    Cdp.configure(api_key_name=os.getenv("CDP_KEY_NAME"), private_key=os.getenv("CDP_PRIVATE_KEY"))

    wallet = Wallet.create(chain)
    if fund:    
        faucet_tx = wallet.faucet('usdc')
        faucet_tx.wait()
        faucet_tx = wallet.faucet('eth')
        faucet_tx.wait()
    return wallet

## Utilities to run FastAPI in a notebook

In [ ]:
#| export
import asyncio, socket, time
from threading import Thread
import uvicorn

In [ ]:
#| export

def is_port_free(port, host='localhost'):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.bind((host, port))
        return True
    except OSError: return False
    finally: sock.close()

def wait_port_free(port, host='localhost', max_wait=3):
    start = time.time()
    while not is_port_free(port):
        if time.time() - start > max_wait: return False
        time.sleep(0.1)
    return True

class ServerManager:
    def __init__(self, app, port=8000, host='0.0.0.0'):
        self.server = uvicorn.Server(uvicorn.Config(app, host=host, port=port))
        
    def start(self):
        Thread(target=lambda: asyncio.run(self.server.serve()), daemon=True).start()
        while not self.server.started: time.sleep(0.01)
        return self
        
    def stop(self):
        if self.server.started:
            self.server.should_exit = True
            wait_port_free(self.server.config.port)
        return True

In [ ]:
#| export
from fastapi import FastAPI
from fastapi.responses import JSONResponse


In [ ]:
# #| export

# def run_l402_server(PRClass, port=9000):
#     app = FastAPI()

#     server = ServerManager(app, port=port).start()

#     @app.get("/offers")
#     def offers():
#         offers_list = [{
#             "amount": 1,
#             "currency": 'USD',
#             "description": 'Purchase 1 credit for API access',
#             "offer_id": 'xxxx',
#             "payment_methods": ['onchain'],
#             "title": '1 Credit Package',
#             type='one-time'
#         )]

        
#         return JSONResponse(
#             content=offers_response.model_dump(),
#             status_code=402
#         )

#     @app.post("/payment_request")
#     async def create_payment_request(request: PRClass):
#         payment_request = ps.create_payment_request(**request.model_dump())
#         return JSONResponse(
#             content=payment_request,
#             status_code=200
#         )